In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

***Класс Cocktail***

*Методы работающие с REST API (www.thecocktaildb.com)*


```
get_random_cocktail()- запрашивает у REST API информацию о случайном   коктейле, возвращает структуру данных с информацией о продукте.

get_cocktail_by_name(name:str) - запрашивает у REST API информацию о кактейле name, возвращает структуру данных с информацией о продукте.
Методы разбирающие полученную структуру данных с информацией о коктейле

show_pic(prod:list) - получает структуру данных, скачивает картинку и возвращает image (200x200)

show_ingr(prod:list) - получает структуру данных, возвращает строку с ингридиентами

show_instr(prod:list) - получает структуру данных, возвращает строку с инструкцией по приготовлению
```

In [2]:
!pip3 install pytelegrambotapi
!pip install -U aiogram
!pip install emoji
!pip install googletrans==3.1.0a0 #==4.0.0-rc1

                                              0.0/232.0 kB ? eta -:--:--
     ---------------                         92.2/232.0 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 232.0/232.0 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=214050 sha256=c03285b0c50e5cfd90a597560bff3e89d7509af3c909c8a57a1fe275d86d2088
  Stored in directory: c:\users\sv_k7\appdata\local\pip\cache\wheels\7e\ba\82\f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pytelegrambotapi
                                              0.0/203.4 kB ? eta -:--:--
     -----------           

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.22.1 requires babel>=2.10, but you have babel 2.9.1 which is incompatible.


                                              0.0/353.7 kB ? eta -:--:--
     ------                                  61.4/353.7 kB 1.7 MB/s eta 0:00:01
     ------------------------               225.3/353.7 kB 2.8 MB/s eta 0:00:01
     -------------------------------------- 353.7/353.7 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350833 sha256=f20647bcd59d22949e2dd7df0c8a07deedc17a0e34ffa91e5a66b36a75e3f06d
  Stored in directory: c:\users\sv_k7\appdata\local\pip\cache\wheels\79\29\1c\234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji


ERROR: Invalid requirement: '#==4.0.0-rc1'


In [3]:
import telebot
from PIL import Image
import requests
import json
from urllib.request import urlopen
from telebot import types
from googletrans import Translator
from emoji import emojize
from aiogram.types import ReplyKeyboardRemove, \
    ReplyKeyboardMarkup, KeyboardButton, \
    InlineKeyboardMarkup, InlineKeyboardButton

class Cocktail:

    def __init__(self):
    
        global url
        url = 'https://www.thecocktaildb.com/api/json/v1/1'

    def get_random_cocktail(self):
        response = requests.get(f"{url}/random.php")
        return response.json()['drinks']

    def get_cocktail_by_name(self,name:str):
        response = requests.get(f"{url}/search.php?s={name}")
        return response.json()['drinks']

    def show_name(self, prod:list):
        if len(prod[0]) != 0:
          return str(f"{prod[0]['strDrink']}\n")
        else: 
          return

    def show_pic(self, prod:list):     
        if len(prod[0]) != 0:
          image = Image.open(urlopen(prod[0]['strDrinkThumb']))
          return image.resize((200,200)) #.show()
        else: 
          return

    def show_ingr(self, prod:list):
       if len(prod[0]) != 0:
         ingr = ''
         for key, value in prod[0].items():
            if (str(key).startswith('strIngredient')) & (str(value)!='None'):
              ingr += str(value)
              ingr += '\n'
         return(ingr)
       else: 
         return
    
    def show_instr(self, prod:list):
       if len(prod[0]) != 0:
         instr=''
         instr=str(f"In {prod[0]['strGlass']}\nUse {prod[0]['strInstructions']}\n")
         return instr
       else: 
         return

if __name__ == '__main__':
# Создание экземпляра кокткйля
  cc = Cocktail()

# Создаем переводчик
trans = Translator()

# ____________ Создаем экземпляр бота ________________

bot = telebot.TeleBot('6250562479:AAHXPPrN9CiFUmGe_pACBeqONaFNQCoHAF0')

# Функция, обрабатывающая команду /start

@bot.message_handler(commands=['start'])
def start(message):

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Да🍸")
    markup.add(btn1)
    
    bot.send_message(message.from_user.id, f"Привет {message.from_user.first_name}, я Коктейлер - бот для подбора напитка. Ну что, приступим? 💃🕺", reply_markup=markup)

@bot.message_handler(commands=['help'])
def welcome_help(message):
    bot.send_message(message.chat.id, 'Цель проекта - создать чат бот, который умеет взаимодействовать с внешними RES API.\
    По запросу пользователя бот на сервисе thecocktaildb находит запрашиваемый напиток, список ингредиентов, рецепт и фото.\
    Рецепт предварительно переводится не русский с помощью сервиса google. Введите /start и следуйте указаниям бота...')    

@bot.message_handler(content_types=['text'])
def get_text_messages(message):

    w_text=["Да🍸","Да","да","Давай","давай","Хорошо","хорошо", "Ок","ок"] # ru

    if (message.text.strip() in w_text):
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      btn1 = types.KeyboardButton("Случайный выбор")
      btn2 = types.KeyboardButton("Я помню название")
      markup.add(btn1, btn2)
      bot.send_message(message.from_user.id, "Сделай свой выбор!🍹", reply_markup=markup)

# Получение сообщений от пользователя

    elif (message.text.strip() == 'Случайный выбор') :
      
      info=''
      
      cocktail= cc.get_random_cocktail()
      
      info=f"<b><i>Name:</i></b>\n{cc.show_name(cocktail)}\n<b><i>Ingredients:</i></b>\n{cc.show_ingr(cocktail)}\n<b><i>Instruction:</i></b>\n{cc.show_instr(cocktail)}<b>ru:</b>\n{trans.translate(cc.show_instr(cocktail), dest='ru').text}"
      
      bot.send_photo(message.chat.id, cc.show_pic(cocktail),info, parse_mode="HTML")
    
       
    elif (message.text.strip() == 'Я помню название') :
        bot.send_message(message.chat.id, 'Введи название коктейля на английском языке, можно несколько букв. Я постараюсь угадать...')
        
    elif cc.get_cocktail_by_name(message.text.strip()):
      
        info=''
  
        cocktail= cc.get_cocktail_by_name(message.text.strip())
  
        info=f"<b><i>Name:</i></b>\n{cc.show_name(cocktail)}\n<b><i>Ingredients:</i></b>\n{cc.show_ingr(cocktail)}\n<b><i>Instruction:</i></b>\n{cc.show_instr(cocktail)}<b>ru:</b>\n{trans.translate(cc.show_instr(cocktail), dest='ru').text}"
      
        bot.send_photo(message.chat.id, cc.show_pic(cocktail),info, parse_mode="HTML")
    
    else: 
        bot.send_message(message.chat.id, 'Коктейль не найден. Введи название коктейля на английском или воспользуйтесь кнопкой "Случайный выбор"')      
      
       
# Запускаем бота

bot.polling(none_stop=True, interval=0)

ModuleNotFoundError: No module named 'googletrans'